In [2]:
## 2019-07-18
# 필요한 module import
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
## 1. Data Loading   / 실제 데이터 정제하는 부분
mnist = input_data.read_data_sets("./data/mnist" , one_hot = True) # csv에 대한 압축파일 저장 / one_hot 처리해야함.


Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [6]:
## 2. Model 정의( Tensorflow graph 생성) /학습하기 위한 그래프그리기 

# tensorflow graph 초기화 / 해당 모델에 대한 그래프
tf.reset_default_graph()

# 데이터를 입력받아서 입력해주는 placeholder
## 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32) # minist가 어떤형태냐에 따라서 정해짐 / dtype은 적지 않으면 기본으로 실수로 표시됨
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
drop_rate = tf.placeholder(dtype=tf.float32) #스칼라니까 데이터 타입만 지정해주면된다.

## 2.2 Convolution = 입력데이터를 좀더 학습이 잘되도록 해주기 위함. 
# CNN은 이미지에 대한 학습에 최적화된 deep learning 방법
# 입력받은 이미지의 형태가 4차원인 매트릭스 / shape를 변경시켜줘야한다.
# ( 이미지의 개수, 이미지의 width, 이미지의 height, color수) / 이미지의 개수: 개수를 알지못하면 -1 / color수 : 1이면 흑백 3이면 컬러
X_img = tf.reshape(X,[-1,28,28,1]) # reshape(어떤걸바꿀지, 어떻게바꿀지) / 어떻게바꿀지 바로위에 설명

## Convolution Layer1 / 특징만 뽑아낸 여러개의 이미지를 만들어내는 작업 => filter를 이용해서
## filter 정의 => filter의 shqpe (width, height, color, filter수(depth))
#filter1 = tf.Variable(tf.random_normal([3,3,1,32])) # 난수발생

## filter를 이용해서 Convolution image를 생성 => activation map 생성
#L1 = tf.nn.conv2d(X_img,filter1,strides=[1,1,1,1], padding="SAME") # nn : neural network
# 이미지 사이즈가 작아지면서 정보들이 유실될수 있다 =>  padding 작업을 하면 방지 가능

## 만들어진 Convolution에 Relu를 적용
# sigmoid를 사용하지 않고 relu를 이용한다 => 값이 0으로 가까워지기 때문에 , 매트릭스가 가지고 있는 값이 커지기 때문에 정제를 해줘야하므로
#L1 = tf.nn.relu(L1)

## pooling 작업(resize, sampling 작업) => optional
#L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME") # ksize : 커널사이즈 , 2x2 앞뒤로 패딩으로 1씩 해준거

## 위에 filter부터 relu,pooling까지 작업을 한번에 할 수 있는 작업 => layers 패키지 이용
L1 = tf.layers.conv2d(inputs=X_img, filters=3, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu) # 3x3 이미지 32개 # kernel_size : 필터의 크기
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2], padding="SAME", strides=2)

## Convolution Layer2
L2 = tf.layers.conv2d(inputs=L1, filters=6, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu) # 3x3 이미지 64개 # kernel_size : 필터의 크기
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
# karas.layers.max_pooling2d 
print(L2.shape) 
# (?, 7, 7, 64)

## 2.3 Neural Network => 2차원을 4차원으로
## Convolution의 결과(4차원)를 Neural Network의 입력(2차원)으로 사용하기 위해 shape를 변경
L2 = tf.reshape(L2,[-1,7*7*6]) # (?, 7, 7, 64) 4차원형태를 2차원으로 변경

## Weight & bias / layer 2개 만들기
W1 = tf.get_variable("weight1", shape=[7*7*6, 256], initializer=tf.contrib.layers.xavier_initializer()) 
# shape=["컬럼수","아웃풋개수"] / shape를 잡아야 행렬곱가능
b1 = tf.Variable(tf.random_normal([256]), name="bias1")
_layer1 = tf.nn.relu(tf.matmul(L2,W1) +b1)
layer1 = tf.layers.dropout(_layer1, rate=drop_rate)

W2 = tf.get_variable("weight2", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer()) 
# shape=["컬럼수","아웃풋개수"] / shape를 잡아야 행렬곱가능
b2 = tf.Variable(tf.random_normal([10]), name="bias2")

## Hypothesis
logits = tf.matmul(layer1,W2) + b2
H = tf.nn.relu(logits)

## Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

## train
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)
# train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

(?, 7, 7, 6)


In [7]:
# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습진행 ( batch 처리 )
training_epoch = 10 # for 루프를 30만큼 돌린다는 의미
batch_size = 100 #  55000개의 행을 다 읽어들이는게 아니라 100개의 행을 읽어서 반복학습!! (아중루프가 돔) / 얼마만큼의 사이즈로 불러드릴껀지

for step in range(training_epoch): # 30 epoch 만큼 반복
    num_of_iter = int(mnist.train.num_examples /  batch_size) # 550번 끊어 읽겠다는 의미
    cost_val = 0
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size) 
        _, cost_val = sess.run([train,cost], feed_dict={X:batch_x , Y:batch_y})
        
    if step % 10 == 0:
        print(cost_val)     

# 학습진행
# for step in range(3000):
#     _, cost_val = sess.run([train,cost], feed_dict={X:mnist.train.images, Y:mnist.train.labels})
    
#     if step % 300 == 0:
#         print(cost_val)

# Accuracy(정확도) 측정
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

result = sess.run([accuracy], feed_dict={X:mnist.test.images, Y:mnist.test.labels})

print("정확도 :{}".format(result))

0.19630119
정확도 :[0.982]


In [ ]:
""" 
    결국 우리 MNIST 예제는 입력한 이미지 1개에 대해 예측한 결과가 H의 값으로 도출
    [0.5,,0.8, 0.99, 0.12, 0.34, ...] 총 10개 / 이중에 가장 큰 값을 몇번 째에 있는 지 찾아내는거
    
"""
### 앙상블(Ensemble) => 모델을 여러개 만들어서 코드처리해야함. / 정확도는 증가하지만 속도는 느려짐(모델이 여러개 이여서)
### 앙상블은 이런 model이 여러개 있다.
### H1 => [0.5,,0.8, 0.99, 0.12, 0.34, ...]
### H2 => [0.2,,0.3, 0.94, 0.5, 0.1, ...]
### H3 => [0.7,,0.1, 0.3, 0.2, 0.12, ...]
### H4 => [0.26,,0.23, 0.194, 0.56, 0.31, ...]

### SUM => [1.66, 1.43, 2.4, 1.3, 1.2, ...] 컬럼별로 값들을 더함
### 최종 prediction은 SUM한 결과값을 가지고 예측 